# Enviar job de Vertex
Enviar job de vertex para el entrenamiento del modelo en cloud

## Consideraciones generales importantes Vertex AI - Noviembre 2023
- El job de entrenamiento que se envia se guarda en el menu principal **"Model Development"**, específicamente en el submenú **"Entrenamiento"**

- Por otro lado, el modelo que queda entrenado queda registrado en el menu principal **"Deploy and Use"**, específicmente en el submenú **"Registro de Modelos"** (solo si se logro ejecutar bien el job y entrenó el modelo. Este funciona como un repositorio de modedlos). Luego de tener registrado el modelo, si se desea, se puede deployar en un endpoint (para realizar predicciones en línea con un delay muy pequeño)  y el endpoint queda registrado en el menu **"EndPoint"**

## Consideraciones para el entrenamiento realizado en este notebook
- Se utiliza la clase **CustomPythonPackageTrainingJob**. Documentación: https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomPythonPackageTrainingJob

- **En este ejemplo, se ENVIA UN PACKAGE CON CÓDIGOS PARA ENTRENAR UN MODELO EN VERTEX. ESTE PACKAGE ESTÁ CONFORMADO POR MÚLTIPLES SCRIPTS. El resto del código funciona de forma igual a los ejemplos v1 y v2, la única diferencia que este ejemplo v3 envia un package en lugar de un script**

- El resto de código (y documentación) igual a los ejemplos v1 y v2. Solo cambia el código para crear el job de entrenamiento (donde se pasa la url de un package .tar.gz de códigos de entrenamiento y el path al script orquestador), y el código para enviar el job es el mismo. Por lo tanto, la documentación repetida se omite y solo se agrega los puntos nuevos para poder utilizar **CustomPythonPackageTrainingJob**


----------------------------------
- **DOCUMENTACIÓN PYTHON DE LA LIBRERÍA COMPLETA AIPLATFORM**: https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform

- **Repo Github oficial de Vertex AI - ejemplos interesantes**:
- https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb
- https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_xgboost.ipynb

In [ ]:
import datetime as dt
import pandas as pd

from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_ID_DS = os.environ.get("PROJECT_GCP", "")

### Paso 0. Parámetros generales

In [ ]:
### PARÁMETROS GENERALES GCP ###
PROJECT_ID = PROJECT_ID_DS
REGION = '{region}'
BUCKET_ID = '{bucket-id}/vertex-ai' # bucket (ya creado) donde se guarda el package .tar.gz con código de entrenamiento

In [ ]:
### PARÁMETROS GENERALES EJECUCIÓN ###

# obtener la hora actual de cuándo se comenzó la ejecución - hash
now = dt.datetime.now()
date_time = now.strftime("%Y_%m_%d_%H_%M_%S")

# identificacion del tipo de caso de uso (y también tipo de modelo) que se va a usar poara registrar el entrenamiento
identity_kind_use_case = 'basic_job_vertex_v3'

# definir path donde se va a guardar el artefacto .pkl del modelo. El path completo se define en el código de entrenamiento ".../model/models.pkl"
BUCKET_ARTIFACT_MODEL = '{bucket-artifact-model}'
path_artifact_model_vertex = f'gs://{BUCKET_ARTIFACT_MODEL}/poc-jobs-vertex/modeltypeB/run_{date_time}/'

In [ ]:
print('-->Parámetros Generales GCP')
print('PROJECT_ID: ', PROJECT_ID)
print('BUCKET_ID: ', BUCKET_ID)
print('REGION: ', REGION)

print('\n------------------------------------------')
print('-->Parámetros Específicos job entrenamiento')
print('date_time: ', date_time)
print('identity_kind_use_case: ', identity_kind_use_case)
print('path_artifact_model_vertex: ', path_artifact_model_vertex)

### Paso 0. Crear scripts de entrenamiento.
Los códigos de entrenamiento (repartidos en múltiples scripts) deben de estar en una carpeta trainer. Esta carpeta debe contener un script **__init__.py** (en blanco) y un script **task.py** el cual es el orquestador y el que se ejecuta al correr el package de entrenamiento; este script task.py llama al resto de scripts auxiliares.

-----------------
**La estructura de carpeta y scripts para generar el package de entrenamiento debe seguir la siguiente estructura recomendada**

- trainer
  - \_\_init\_\_.p
  - taskpy.
  - script_aux1.py
  - script_aux2.py
  - script_auxN.pypypy

In [ ]:
# ESCRIBIR EL SCRIPT DE ENTRENAMIENTO.

In [ ]:
# # Correr script de entrenamiento de forma local - solo verificar que no hay errores
# %run src/trainer/task.py --id_date_time 2023_10_29_02_03_38

### Paso 1. Crear package con los códigos de entrenamiento
Luego de tener creados los scripts de entrenamiento estructurados en el formato recomendado y probados que corren en el local (probados por ejemplo con una submuestra de datos), crear el package de entrenamiento.

**IMPORTANTE: OBLIGATORIAMENTE EL FORMATO DE CARPETAS QUE ACEPTA VERTEX ES "src/trainer/task.py". Se necesitan 2 carpetas y luego llegar al script task.py**

------------------
**Ejemplo de muestra (subir package con comandos de linux)** (noviembre 2023): https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/get_started_with_vertex_distributed_training.ipynb

Fuente: https://packaging.python.org/en/latest/tutorials/packaging-projects/

------------------
**Para crear el package.tar.gz con los scripts de entrenamiento se necesitan la siguiente estructura de carpetas:**

- src
  - trainer
  - trainer_lr.egg-info
- packages-pip
- setup.py

Donde:
- **setup.py:** Es un script que se escribe manualmente y es necesario para crear el package.tar.gz de entrenamiento. Aquí se indican la lista de requirements de los packages que tienen que instalarse para correr el código
- src: Carpeta source/src. En su interior se ubica la carpeta trainer la cual contiene todos los códigos de entrenamiento
- packages-pip: carpeta que se crea automáticamente, en esta se guardan los packages .tar.gz

In [ ]:
##### DEFINIR PARÁMETROS UTILIZADOS LA CREACIÓN DEL ARCHIVO SETUP.PY Y DE LA SUBIDA DEL PACKAGE .TAR.GS A GCS PARA EL ENTRENAMIENTO DEL MODELO

#------------
# requirements
list_requirements_setup = ["google-cloud-bigquery==3.11.4", "db-dtypes", "gcsfs==2023.9.2", "pandas==2.0.3", "numpy==1.23.5", "scikit-learn==1.3.1"]

# version package - definir con doble string
name_package_setup = "trainer-model"
version_setup = "1.0.0"

#string solo para poder que funciona el f string. Para que funcione el script setup.py: package_dir={'': 'src'},
package_dir_setup = eval('{"": "src"}')

#------------
# definir folder local donde se guardan los archivo .tar.gz de los packages generados
path_folder_package = 'packages-pip'


#------------
# path local y gcs del package con los codigos de entrenamiento .tar.gz
path_train_package_local = f'{path_folder_package}/{name_package_setup}-{version_setup}.tar.gz'
path_train_package_gcs = f'gs://{BUCKET_ID}/package-{name_package_setup}-{date_time}.tar.gz'

In [ ]:
#### ESCRIBIR SETUP.PY
# Se genera un docstring con el código que debería ir en el script setup.py y luego se guarda el string en dicho formato

string_setup_py = f"""import setuptools
setuptools.setup(
    name='{name_package_setup}',
    version='{version_setup}',
    author='Jose',
    description='job vertex v3 - enviar package custom de entrenamiento',
    install_requires = {list_requirements_setup},
    package_dir={package_dir_setup},
    packages=setuptools.find_packages(where="src"),
    include_package_data=True
)
"""

with open('setup.py', 'w') as file:
    file.write(string_setup_py)

In [ ]:
##### SUBIR PACKAGE .TAR.GZ A GCS PARA ENTRENAMIENTO DEL MODELO

# ejecutar sctript setup.py y generar package .tar.gz guardado localmente (path_folder_package: folder donde se guarda lolcamente)
! python setup.py sdist --formats=gztar -d $path_folder_package

# teniendo el package .tar.gz guardado localmente, subirlo a GCS para realizar el entrenamiento del modelo
! gsutil cp $path_train_package_local $path_train_package_gcs

### Paso 2: Inicializar Vertex AI

In [ ]:
aiplatform.init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_ID)

### Paso 3. Definir parámetros necesarios para CREAR la instancia del job de entrenamiento (aún no se envia)

---------------------------
- Para entrenar un modelo en cloud se deben realizar 2 pasos: el primero crear la instancia de la clase del entrenamiento (CustomPythonPackageTrainingJob) y en segundo lugar enviar el job de entrenamiento (método de la instancia)

---------------------------
- INFO, LOS ÚNICOS PARÁMETROS QUE SON OBLIGATORIOS SON:
    - **display_name**: nombre del job
    - **python_package_gcs_uri**: package de python donde están los códigos de entrenamiento + requirements
    - **python_module_name**: script orquestador de python dentro de package de entrenamiento
    - **container_uri**: container de entrenamiento. preferencia utilizar container prebuild de GCP e instalar los packages faltantes de requirements

---------------------------
- INFO containers base de google:
    - **Container para el entrenamiento**: https://cloud.google.com/vertex-ai/docs/training/pre-built-containers
    - **Container para la predicción**: https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

In [ ]:
### definir el nombre del job que se enviará. Algo que indentifique de qué es el job + hora envio ###
job_name = identity_kind_use_case + '__job_train__' + date_time
job_name

In [ ]:
### definir el contrainer para el ENTRENAMIENTO y para LA PREDICCIÓN - facilitados por google ####
container_train = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest'
container_deploy = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest'

In [ ]:
### definir el path a package de entrenamiento en GCS y al script orquestador ###

print('path train package gcs: ', path_train_package_gcs)

python_script_orchestrator = 'trainer.task' # no se agrega el .py
print('python_script_orchestrator: ', python_script_orchestrator)

In [ ]:
### definir la descripción del modelo ###
description = 'entrenar modelo utilizando "CustomPythonPackageTrainingJob"'

### Paso 4. Definir parámetros necesarios para ENVIAR job de entrenamiento - usando CPU
- Igual que el ejemplo v2. Pasar args al script orquestador del entrenamiento y guardar artefacto pkl del modelo en un path custom de GCS y registrarlo en modelos Vertex

In [ ]:
### definir el nombre con el que queda registrado (en VERTEX AI) el modelo resultado del entrenamiento ###
# De qué es el modelo +  hora de envio
model_name = identity_kind_use_case  + '__model__' + date_time 
model_name

In [ ]:
### definir el tipo de máquina para hacer el entrenamiento ###
machine_type_train = "n1-standard"
vcpu_train = "4"
train_compute = machine_type_train + "-" + vcpu_train

print("Train machine type: ", train_compute)

### Paso 5. Crear instancia del job de entrenamiento a VERTEX AI (CustomTrainingJob)
- Define your custom TrainingPipeline on Vertex AI.
- Use the **CustomPythonPackageTrainingJob** to define the TrainingPipeline.

In [ ]:
# PRIMERO SE LLAMA UNA INSTANCIA DE LA CLASE
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name = job_name,
    python_package_gcs_uri = path_train_package_gcs,
    python_module_name = python_script_orchestrator,
    container_uri = container_train,

    model_description = description,
    model_serving_container_image_uri = container_deploy,
)

In [ ]:
job

### Paso 6. Enviar el job de entrenamiento a VERTEX AI (CustomTrainingJob)
- Solo basta con cambiar la clase CustomPythonPackageTrainingJob (en comparación con ejemplos v1 y v2) al definir el job y los parámetros de este. **Luego de tener creado el job de vertex, para enviar al entrenamiento (job.run) no es necesario ninguna modificación**

In [ ]:
model = job.run(
    model_display_name = model_name,
    replica_count = 1,
    machine_type = train_compute,
    base_output_dir = path_artifact_model_vertex, # path custom .../model/model.pkl donde se guarda el pkl del modelo. se omite del path model/model.pkl
    args = ["--id_date_time=" + date_time], # args que se le pasan al script de entrenamiento de este ejemplo
    sync = True
)

In [ ]:
model